In [8]:
import warnings
warnings.filterwarnings("ignore")

import env
import wrangle

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
# modeling methods
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import f_regression 

np.set_printoptions(suppress=True)

In [9]:
df = pd.read_csv("clean_zillow.csv")

In [10]:
train, validate, test = wrangle.split_zillow(df)

In [11]:
X_train, X_validate, X_test, y_train, y_validate, y_test = wrangle.split_train_validate_test(train, validate, test)

# Baseline

In [12]:
# We need y_train and y_validate to be dataframes to append the new columns with predicted values. 
y_train = pd.DataFrame(y_train)
    # turn it into a single pandas dataframe
y_validate = pd.DataFrame(y_validate)
    # wrap them as dataframes

# 1. Predict appraised_value_pred_mean
    # 2 different aselines of mean and medium
logerror_pred_mean = y_train['logerror'].mean()
y_train['logerror_pred_mean'] = logerror_pred_mean
y_validate['logerror_pred_mean'] = logerror_pred_mean

# 2. compute appraised_value_pred_median
    # same process as mean (above)
logerror_pred_median = y_train['logerror'].median()
y_train['logerror_pred_median'] = logerror_pred_median
y_validate['logerror_pred_median'] = logerror_pred_median

# 3. RMSE of appraised_value_pred_mean
rmse_train_mean = mean_squared_error(y_train.logerror, 
                                     y_train.logerror_pred_mean)**(1/2)
    # stick with root mean square error
        # not your only option but that is what we will be using here
            # just because it is eaiest to us and explain
    # remember when you call you it will be your y_true and y_pred
rmse_validate_mean = mean_squared_error(y_validate.logerror, 
                                        y_validate.logerror_pred_mean)**(1/2)
    # do the same thing for the validate set as done above for the train set
    
print("RMSE using Mean\nTrain/In-Sample: ", round(rmse_train_mean, 2), 
      "\nValidate/Out-of-Sample: ", round(rmse_validate_mean, 2))
print(' ')
# 4. RMSE of appraised_value_pred_median
rmse_train_medium = mean_squared_error(y_train.logerror, 
                                       y_train.logerror_pred_median)**(1/2)

rmse_validate_medium = mean_squared_error(y_validate.logerror, 
                                          y_validate.logerror_pred_median)**(1/2)

print("RMSE using Median\nTrain/In-Sample: ", round(rmse_train_medium, 2), 
      "\nValidate/Out-of-Sample: ", round(rmse_validate_medium, 2))

# median comes out a little higher than mean 
    # but a little lower than our validation set

RMSE using Mean
Train/In-Sample:  0.16 
Validate/Out-of-Sample:  0.17
 
RMSE using Median
Train/In-Sample:  0.16 
Validate/Out-of-Sample:  0.17
